# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
def prefilter_items(data, items, take_n_popular=5000):
    # Уберем самые популярные товары (их и так купят)
    popularity = pd.DataFrame(data.groupby('item_id')['user_id'].nunique() / data['user_id'].nunique()).reset_index()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data.loc[data['item_id'].isin(top_popular), 'item_id'] = 999999

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data.loc[data['item_id'].isin(top_notpopular), 'item_id'] = 999999

    # Уберем товары, которые не продавались за последние 12 месяцев
    day = data.groupby('item_id')['day'].max().reset_index()
    day = day[day['day'] < day['day'].max() - 365].item_id.tolist()
    data.loc[data['item_id'].isin(day), 'item_id'] = 999999

    # Уберем не интересные для рекоммендаций категории (department)

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    data['cost'] = data['sales_value'] / data['quantity']
    np.nan_to_num(data['cost'], copy=False, nan=0.0, posinf=0.0, neginf=0.0)

    costs = pd.DataFrame(data.groupby('item_id')['cost'].mean()).reset_index()
    low_cost = costs[costs['cost'] <= 1].item_id.tolist()

    data.loc[data['item_id'].isin(low_cost), 'item_id'] = 999999

    # Уберем слишком дорогие товары
    high_cost = costs[costs['cost'] > 100].item_id.tolist()
    data.loc[data['item_id'].isin(high_cost), 'item_id'] = 999999
    
    data = data.drop('cost', axis=1)

    # Top-N
    popularity = popularity[popularity['share_unique_users'] <= 0.5].sort_values('share_unique_users', ascending=False)
    top = popularity[popularity['item_id'].isin(data['item_id'])].item_id.head(take_n_popular).values
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    return data

In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Diana\AppData\Local\Temp\ipykernel_692\1967547428.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cost'] = data['sales_value'] / data['quantity']


Decreased # items from 86865 to 5001


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,818981,819063,819255,819304,819308,819330,819518,819594,819765,819840,...,15925334,15926712,15926775,15926844,15926863,15926885,15926886,15926887,15926927,15972074
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),  # На вход item-user matrix
          show_progress=True)

C:\Users\Diana\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 4.03 s


In [11]:
def get_recommendations(user, sparse_user_item, model, N=5):
    res = [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item[userid_to_id[user]],   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]]
    return res

In [12]:
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, sparse_user_item=sparse_user_item, model=model, N=5))
result.head(3)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1051211, 1025641, 934369, 1082212, 1056509]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1133018, 1106523, 1053690, 1092026, 951590]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[895268, 857006, 1051516, 1023720, 1024306]"


-----

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [13]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    weighting: string
        Тип взвешивания, один из вариантов None, "bm25", "tfidf"
    fake_id: int
        идентификатор, которым заменялись редкие объекты, можно передать None, если такого объекта нет
    """
    
    def __init__(self, data, weighting=None, fake_id=None):
        
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != fake_id]
        
        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        if fake_id is not None:
            self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != fake_id]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        # Дальнейшая предобработка
        self.fake_id = fake_id
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
        self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        self.user_item_matrix = csr_matrix(self.user_item_matrix).tocsr()
        self.user_item_matrix_for_pred = self.user_item_matrix
        
        if weighting == 'bm25':
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T.tocsr()
        elif weighting == 'tfidf':
            self.user_item_matrix = tfidf_weight(self.user_item_matrix.T).T.tocsr()
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(user_item_matrix)
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(user_item_matrix)
        
        return model

    def update_dict(self, user_id):
        """Если появился новый user / item, то нужно обновить словари"""
        if user_id not in self.userid_to_id.keys():
            max_id = max(list(self.userid_to_id.values()))
            max_id += 1
            
            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})
    
    def get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2) # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[0][1] # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоменндаций < N, то дополняем их топ-популярными"""
        if len(recommendations) < N:
            top_popular = [rec for rec in self.overall_top_purchases[:N] if rec not in recommendations]
            recommendations.extend(top_popular)
            recommendations = recommendations[:N]
        return recommendations
    
    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стандартные библиотеки implicit"""
        self.update_dict(user_id=user)
        filter_items = [] if self.fake_id is None else [self.itemid_to_id[self.fake_id]]
        res = model.recommend(userid=self.userid_to_id[user],
                             user_items=self.user_item_matrix_for_pred[self.userid_to_id[user]],
                             N=N,
                             filter_already_liked_items=False,
                             filter_items=filter_items,
                             recalculate_user=True)
        mask = res[1].argsort()[::-1]
        res = [self.id_to_itemid[rec] for rec in res[0][mask]][:N]
        res = self.extend_with_top_popular(res, N=N)
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стандартные библиотеки implicit"""
        self.update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)
    
    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""
        self.update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)
    
    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        
        top_items = self.top_purchases[self.top_purchases['user_id'] == user].head(N).item_id.values
        similar_items = [self.get_similar_item(x) for x in top_items]
        similar_items = self.extend_with_top_popular(similar_items, N=N)

        assert len(similar_items) == N, 'Количество рекомендаций != {}'.format(N)
        return similar_items
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)[0][1:]
        similar_users = [self.id_to_userid[u] for u in similar_users]
        top_items = list(self.top_purchases[self.top_purchases['user_id']\
                                       .isin(similar_users)].head(N).item_id)
        top_items = self.extend_with_top_popular(top_items, N=N)

        assert len(top_items) == N, 'Количество рекомендаций != {}'.format(N)
        return top_items

In [14]:
recommender = MainRecommender(data_train, weighting='tfidf', fake_id=999999)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [15]:
recommender.get_als_recommendations(user=2500)

[1058997, 1126899, 961554, 916122, 859075]

In [16]:
recommender.get_similar_items_recommendation(user=2375)

[868764, 1106523, 1044078, 1046545, 12385373]

In [17]:
result['als'] = result['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=5))
result.head(3)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[867188, 1100972, 1056509, 5569374, 1005186]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5569230, 1106523, 1053690, 951590, 8090521]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1024306, 965267, 930118, 1023720, 878996]"


----

Проверка, что все работает

In [18]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender